In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import pickle

In [2]:
# file = open('googleNet_embeddings.pkl', 'rb')
embeddings = torch.load('../matrix_factorization/InceptionNet_embeddings.pkl')
embeddings

{'E0': tensor([[ 0.5170, -0.4419, -0.2317,  ..., -0.2739, -0.0532,  0.0989],
         [ 0.4730, -0.0626, -0.4108,  ..., -0.3816, -0.0724, -0.1168],
         [-0.0658, -0.4444, -0.8215,  ...,  0.0046, -0.2929,  0.5539],
         ...,
         [ 0.3002,  0.0253,  0.0117,  ..., -0.1760, -0.3189, -0.2569],
         [ 0.0696, -0.2516, -0.8603,  ..., -0.4519, -0.2494,  0.1623],
         [ 0.1368, -0.6130, -0.5538,  ..., -0.7279, -0.4489,  0.0234]],
        requires_grad=True),
 'E1': tensor([[ 0.0148, -0.0759,  0.0269,  ..., -0.0563,  0.0771, -0.1214],
         [-0.0219, -0.0221,  0.0357,  ..., -0.0592,  0.0780, -0.0646],
         [-0.0220, -0.0605,  0.1777,  ...,  0.1040, -0.0823, -0.0530],
         ...,
         [-0.0285, -0.0469,  0.0993,  ...,  0.0194,  0.0008, -0.0932],
         [-0.0172, -0.0372,  0.0263,  ..., -0.0611,  0.0687, -0.1000],
         [ 0.1997, -0.1324, -0.0375,  ..., -0.0423,  0.1198, -0.2128]],
        requires_grad=True),
 'E2': tensor([[ 0.0982, -0.6205,  0.0477,  ...,

In [3]:
train_ids = torch.load("../dataset/train_book_id.pkl")
test_ids = torch.load("../dataset/test_book_id.pkl")
print(train_ids.size())
print(test_ids.size())

torch.Size([4000])
torch.Size([1000])


In [4]:
df = pd.read_csv("../dataset/books_with_genres/books_with_genres.csv")
df.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,"fantasy, paranormal","mystery, thriller, crime",poetry,romance,non-fiction,children,young-adult,"comics, graphic",category,binary_category
0,NaN,7,['189911'],US,eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,False,4.03,NaN,...,31.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,"fantasy, paranormal",fiction
1,0842379428,566,[],US,eng,"[{'count': '6393', 'name': 'to-read'}, {'count...",NaN,False,4.26,B000FCKCJC,...,NaN,NaN,NaN,NaN,163.0,NaN,NaN,NaN,non-fiction,non-fiction
2,0590417010,193,[],US,eng,"[{'count': '450', 'name': 'to-read'}, {'count'...",NaN,False,4.43,B017RORXNI,...,NaN,NaN,NaN,NaN,2.0,109.0,1.0,NaN,children,non-fiction
3,1400041694,44,[],US,en-US,"[{'count': '362', 'name': 'to-read'}, {'count'...",NaN,False,3.75,B002OTKEP6,...,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,non-fiction,non-fiction
4,NaN,1,[],US,eng,"[{'count': '8', 'name': 'to-read'}, {'count': ...",NaN,False,3.83,NaN,...,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,poetry,non-fiction


In [5]:
X_train = embeddings["E0"][0:4000].detach().numpy()
X_test = embeddings["E0"][4000:].detach().numpy()
print(X_train.shape)
print(X_test.shape)

(4000, 50)
(1000, 50)


In [6]:
y_train = df["average_rating"][0:4000]
y_test = df["average_rating"][4000:]
print(y_train.shape)
print(y_test.shape)

(4000,)
(1000,)


In [7]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
svr = SVR()
svr.fit(X_train, y_train)
preds = svr.predict(X_test)
error = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print('error: ', error)
print('r2 score: ', r2)

error:  0.18598370323110902
r2 score:  -0.06665659825387271


In [8]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
preds = lr.predict(X_test)
error = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print('error: ', error)
print('r2 score: ', r2)

error:  0.17637152527023972
r2 score:  -0.011528687219586331


In [9]:
from sklearn.neural_network import MLPRegressor
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
preds = regr.predict(X_test)
error = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print('error: ', error)
print('r2 score: ', r2)

error:  0.2791426496795285
r2 score:  -0.6009432222389184


In [10]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
regr = BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=10, random_state=0).fit(X_train, y_train)
preds = regr.predict(X_test)
error = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
print('error: ', error)
print('r2 score: ', r2)

error:  0.19256091600000003
r2 score:  -0.10437833019153286
